<a href="https://colab.research.google.com/github/DanielDialektico/rag_agentes_langchain_curso/blob/main/notebooks/langchain_rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://dialektico.com/wp-content/uploads/2023/03/MiniLogoW4.png" alt="Dialéktico Logo" />

Este pequeño tutorial pertenece al curso de RAG y agentes con LangChain al que puedes acceder mediante la siguiente URL: https://www.youtube.com/playlist?list=PLlWTv9_GeWd32stuEMWpYOnxiVxnXaU6q

Sigue los videos del curso para recibir instrucciones y contexto sobre la ejecución de este Notebook.

<br>

# Se instalan e importan las librerías

In [ ]:
# Se instalan las librerías.
!pip install langchain-pinecone==0.2.4
!pip install langchain==0.3.21
!pip install langchain-community==0.3.20
!pip install beautifulsoup4==4.13.3
!pip install tiktoken==0.9.0
!pip install pypdf==5.4.0
!pip install pinecone==6.0.2
!pip install langchain-huggingface==0.1.2
!pip install langchain_deepseek==0.1.2

In [ ]:
# Se importan las librerías.
import os
import warnings
import sys
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_community.document_loaders import PyPDFLoader, WebBaseLoader
from langchain_text_splitters import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain_pinecone import PineconeVectorStore
from langchain_deepseek import ChatDeepSeek
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.schema import BaseChatMessageHistory, BaseMessage
import bs4
from google.colab import userdata
from pinecone import Pinecone, ServerlessSpec
from uuid import uuid4
from IPython.display import display, HTML
from pydantic import BaseModel, Field
from typing import List


warnings.filterwarnings('ignore')

<br>

# Vectorización de documentos y almacenamiento.

In [ ]:
# Se carga la información desde distintas fuentes
web_loader = WebBaseLoader(web_paths=["https://dialektico.com/cama-ultra-lujosa-para-gatos-dialektiroyal-comfort/"])
documents = []

for doc in os.listdir('documents'):
  pdf_loader = PyPDFLoader('documents/'+ doc)
  async for page in pdf_loader.alazy_load():
      documents.append(page)

async for doc in web_loader.alazy_load():
    documents.append(doc)

In [ ]:
# Cantidad de documentos.
len(documents)

In [ ]:
# Se prepara el algoritmo de embeddings a utilizar.
embeddings_model = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

In [ ]:
# Se crea vector store en memoria para almacenar vectores.
vector_store = InMemoryVectorStore(embedding=embeddings_model)

# Se instancia el CharacterTextSplitter.
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    encoding_name="cl100k_base", chunk_size=1000, chunk_overlap=200
)

# Se divide el texto de los documentos utilizando el tokenizador tiktoken.
chunks = text_splitter.split_documents(documents)

# Se almacenan los vectores.
vector_store.add_documents(documents=chunks, ids=[str(uuid4()) for _ in range(len(chunks))])

# RAG

In [ ]:
# Se añade la API key como variable de ambiente desde un secreto en Colab.
os.environ["DEEPSEEK_API_KEY"] = userdata.get('DEEPSEEK_API_KEY')

# Se define el modelo a utilizar y añaden valores de parámetros.
model = ChatDeepSeek(
      model="deepseek-chat",
      temperature=0,
      max_tokens=200
      )

In [ ]:
# Se crea template para RAG.
template = """Utiliza los siguientes fragmentos de contexto para responder la
pregunta al final. Si no sabes la respuesta, simplemente di que no la sabes, no
intentes inventar una respuesta.
Utiliza pocas oraciones y mantén la respuesta lo más concisa posible.

CONTEXTO: {context}

PREGUNTA: {question}

Respuesta útil:"""
prompt = PromptTemplate.from_template(template)

message = prompt.invoke(
    {"context": "(Se añade el contexto)", "question": "(Se añade la pregunta)"}
).to_messages()

print(message[0].content)

# Recuperación (retrieve)

In [ ]:
# Se traen documentos con mayor similaridad.
question = "¿Qué incluye la cama ultralujosa para gatos?"
retrieved_docs = vector_store.similarity_search(question, k=5)
docs_content = "\n\n".join(doc.page_content for doc in retrieved_docs)

In [ ]:
docs_content

## Incremento (Augment)

In [ ]:
# Se añade la información al prompt.
messages = prompt.invoke({"question": question, "context": docs_content})

In [ ]:
print(messages)

## Generación (Generation)

In [ ]:
# Se genera la respuesta con el contexto dado.
response = model.invoke(messages)
response.content

<br>

Se encapsula la lógica en una sola función para crear un chat.

In [ ]:
def run_rag_chat(vector_store, model):

    # Se define el template
    prompt_template = """Utiliza los siguientes fragmentos de contexto para responder la
    pregunta al final. Si no sabes la respuesta, simplemente di que no la sabes, no
    intentes inventar una respuesta.
    Utiliza pocas oraciones y mantén la respuesta lo más concisa posible.

    CONTEXTO: {context}

    PREGUNTA: {question}

    Respuesta útil:"""

    print("\n📝 Escribe una pregunta (o escribe 'salir' para terminar):")

    while True:
        try:
            user_question = input("\nPregunta: ").strip()

            if user_question.lower() in ('salir', 'exit', 'quit'):
                print("\n🚪 Saliendo del chat. ¡Hasta luego!")
                break

            # Se recuperan documentos similares.
            retrieved_docs = vector_store.similarity_search(user_question, k=5)
            docs_content = "\n\n".join(doc.page_content for doc in retrieved_docs)

            # Se añade el contexto al prompt.
            prompt = PromptTemplate.from_template(prompt_template)
            messages = prompt.invoke({"question": user_question, "context": docs_content})

            # Se genera la respuesta.
            response = model.invoke(messages)

            # Se muestra la respuesta.
            display(HTML(f"<div style='padding:10px; border:1px solid #ccc; border-radius:8px;'><b>Respuesta:</b><br>{response.content}</div>"))

            print("\n📝 Escribe otra pregunta (o 'salir' para terminar):")

        except KeyboardInterrupt:
            print("\n\n🛑 Interrupción manual detectada. Cerrando chat.")
            break
        except Exception as e:
            print(f"\n❗ Error: {e}")
            break

In [ ]:
run_rag_chat(vector_store, model)

<br>

Se añade memoria y streaming al chat.

In [ ]:
def run_rag_chat_memory(vector_store, model):

    # Definir historial de mensajes en memoria.
    class InMemoryHistory(BaseChatMessageHistory, BaseModel):
        messages: List[BaseMessage] = Field(default_factory=list)

        def add_messages(self, messages: List[BaseMessage]) -> None:
            self.messages.extend(messages)

        def clear(self) -> None:
            self.messages = []

    store = {}

    def get_by_session_id(session_id: str) -> BaseChatMessageHistory:
        if session_id not in store:
            store[session_id] = InMemoryHistory()
        return store[session_id]

    # Definir el template dentro de la función
    prompt_template = """Utiliza los siguientes fragmentos de contexto para responder la
    pregunta al final. Si no sabes la respuesta, simplemente di que no la sabes, no
    intentes inventar una respuesta.
    Utiliza pocas oraciones y mantén la respuesta lo más concisa posible.

    CONTEXTO: {context}

    PREGUNTA: {question}

    Respuesta útil:"""

    # Se genera un un session_id dinámico.
    session_id = str(uuid4())
    print(f"\n🆔 Sesión iniciada con ID: {session_id}")
    print("📝 Escribe una pregunta (o escribe 'salir' para terminar):")

    while True:
        try:
            user_question = input("\nPregunta: ").strip()

            if user_question.lower() in ('salir', 'exit', 'quit'):
                print("\n🚪 Saliendo del chat. ¡Hasta luego!")
                break

            # Se recuperan documentos similares.
            retrieved_docs = vector_store.similarity_search(user_question, k=5)
            docs_content = "\n\n".join(doc.page_content for doc in retrieved_docs)

            # Crear el prompt compuesto
            chat_prompt = ChatPromptTemplate.from_messages([
                ("system", prompt_template),
                MessagesPlaceholder(variable_name="history"),
                ("user", "{question}"),
            ])

            # Encadenar prompt con modelo
            chain = chat_prompt | model

            # Añadir historial
            chain_with_history = RunnableWithMessageHistory(
                chain,
                get_by_session_id,
                input_messages_key="question",
                history_messages_key="history"
            )

            # Se aumenta y genera la respuesta, mostrándose con streaming.
            print("\n💬 Respuesta:")
            chunks = []
            for chunk in chain_with_history.stream(
                {"question": user_question, "context": docs_content},
                config={"configurable": {"session_id": session_id}}
            ):
                chunks.append(chunk)
                print(chunk.content, end="", flush=True)

            print("\n\n📝 Escribe otra pregunta (o 'salir' para terminar):")

        except KeyboardInterrupt:
            print("\n\n🛑 Interrupción manual detectada. Cerrando chat.")
            break
        except Exception as e:
            print(f"\n❗ Error: {e}")
            break

In [ ]:
run_rag_chat_memory(vector_store, model)

In [ ]:
# Dialektico Machine learning practices © 2025 by Daniel Antonio García Escobar
# is licensed under CC BY-NC 4.0. To view a copy of this license,
# visit https://creativecommons.org/licenses/by-nc/4.0/

# Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International
# Public License